# Trabalho final


**Autores:**

Francisco de Assis Marinho Aguiar


Leandro Da Cruz Farias

## Importação das biblotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost

from sklearn.datasets import load_wine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, SCORERS
from sklearn.metrics import average_precision_score
from operator import itemgetter

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist
from keras.models import Sequential
from keras.layers import Flatten, Dense

### Configuração das bibliotecas

In [ ]:
pd.options.display.max_columns = None

sns.set(font_scale=1.4)
sns.set_style({'font.family': 'serif', 'fontname': 'Times New Roman'})

mpl.rcParams['figure.dpi'] = 100

## Conjunto de dados

### Dados de treino

In [ ]:
!wget -O training_dataset.csv https://drive.google.com/u/0/uc?id=1sQei4yJqUKDrB-EHdHW2ZN3hsO3EqJCS&export=download

--2021-06-25 00:06:36--  https://drive.google.com/u/0/uc?id=1sQei4yJqUKDrB-EHdHW2ZN3hsO3EqJCS
Resolving drive.google.com (drive.google.com)... 172.217.12.238, 2607:f8b0:4004:807::200e
Connecting to drive.google.com (drive.google.com)|172.217.12.238|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-98-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/55s6sjcqlu1igstotkqcer6ku0h8ckfd/1624579575000/12797427796612289247/*/1sQei4yJqUKDrB-EHdHW2ZN3hsO3EqJCS [following]
--2021-06-25 00:06:39--  https://doc-0s-98-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/55s6sjcqlu1igstotkqcer6ku0h8ckfd/1624579575000/12797427796612289247/*/1sQei4yJqUKDrB-EHdHW2ZN3hsO3EqJCS
Resolving doc-0s-98-docs.googleusercontent.com (doc-0s-98-docs.googleusercontent.com)... 172.217.2.97, 2607:f8b0:4004:80a::2001
Connecting to doc-0s-98-docs.googleusercontent.com (doc-0s-98-docs.googleusercontent.com)|172.217.2.97|

In [ ]:
training_df = pd.read_csv('training_dataset.csv')
training_df.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,120.0,38.0,120.0

### Dados de teste

In [ ]:
!wget -O test_dataset.csv https://drive.google.com/u/0/uc?id=1NKzd1DMTs93QK1_yoVmdnIOEReuGJcKn&export=download

--2021-06-25 00:06:42--  https://drive.google.com/u/0/uc?id=1NKzd1DMTs93QK1_yoVmdnIOEReuGJcKn
Resolving drive.google.com (drive.google.com)... 172.217.15.78, 2607:f8b0:4004:807::200e
Connecting to drive.google.com (drive.google.com)|172.217.15.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-98-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fdq41ik5ogisjsm4d8b9s7s6c2qsgnr4/1624579575000/12797427796612289247/*/1NKzd1DMTs93QK1_yoVmdnIOEReuGJcKn [following]
--2021-06-25 00:06:43--  https://doc-0k-98-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fdq41ik5ogisjsm4d8b9s7s6c2qsgnr4/1624579575000/12797427796612289247/*/1NKzd1DMTs93QK1_yoVmdnIOEReuGJcKn
Resolving doc-0k-98-docs.googleusercontent.com (doc-0k-98-docs.googleusercontent.com)... 172.217.2.97, 2607:f8b0:4004:80a::2001
Connecting to doc-0k-98-docs.googleusercontent.com (doc-0k-98-docs.googleusercontent.com)|172.217.2.97|:4

In [ ]:
test_df = pd.read_csv('test_dataset.csv')
test_df.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,206833,19,37.0,27.109375,0,Caucasian,F,160.0,Floor,Floor,653,admit,Med-Surg ICU,4.277083,0,69.4,NaN,109.0,108.01,0,0,NaN,30.0,1.56,NaN,4.0,6.0,0.0,5.0,84.0,118.0,23.3,0,128.0,NaN,NaN,NaN,NaN,6.0,134.0,36.5,NaN,0,25.46,NaN,NaN,116.0,77.0,116.0,77.0,112.0,75.0,NaN,NaN,128.0,86.0,128.0,86.0,20.0,6.0,99.0,91.0,NaN,NaN,165.0,116.0,165.0,116.0,37.16,36.5,NaN,NaN,109.0,99.0,109.0,99.0,107.0,102.0,NaN,NaN,120.0,112.0,120.0,112.0,19.0,12.0,98.0,97.0,NaN,NaN,157.0,149.0,157.0,149.0,36.61,36.61,NaN,NaN,NaN,NaN,30.0,30.0,NaN,NaN,1.56,1.56,112.0,84.0,19.0,19.0,9.2,7.7,28.3,23.3,1.1,1.0,NaN,NaN,236.0,203.0,5.2,5.2,134.0,134.0,25.46,24.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1,273200,86,47.0,28.071089,1,Caucasian,M,164.0,Operating Room,Operating Room / Recovery,1035,admit,CCU-CTICU,0.340278,0,75.5,NaN,308.0,1207.01,1,0,NaN,12.0,0.65,0.8,4.0,4.0,0.0,1.0,173.0,93.0,28.6,1,51.0,45.0,45.0,95.0,7.29,28.0,137.0,36.4,3902.2560,1,13.90,68.0,42.0,64.0,49.0,64.0,49.0,91.0

## Tratamento dos dados

In [ ]:
def categorize(df):
    df['ethnicity'] = pd.Categorical(df['ethnicity'])
    df['ethnicity'] = df['ethnicity'].cat.codes

    df['gender'] = pd.Categorical(df['gender'])
    df['gender'] = df['gender'].cat.codes

    df['hospital_admit_source'] = pd.Categorical(df['hospital_admit_source'])
    df['hospital_admit_source'] = df['hospital_admit_source'].cat.codes

    df['icu_admit_source'] = pd.Categorical(df['icu_admit_source'])
    df['icu_admit_source'] = df['icu_admit_source'].cat.codes

    df['icu_stay_type'] = pd.Categorical(df['icu_stay_type'])
    df['icu_stay_type'] = df['icu_stay_type'].cat.codes

    df['icu_type'] = pd.Categorical(df['icu_type'])
    df['icu_type'] = df['icu_type'].cat.codes

def normalize(df):
  for column in df.columns[1:]:
    df[column] = pd.to_numeric(df[column], errors='coerce')
    new_value = (df[column] - df[column].min())/(df[column].max() - df[column].min())
    df[column] = new_value

def remove_nan(df):
  
  for column in df.columns:
    if (df[column].isna().sum() > 0):
         df[column] = df[column].fillna(0)

### Treino

In [ ]:
categorize(training_df)
normalize(training_df)
remove_nan(training_df)

In [ ]:
training_df.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,214826,0.576355,0.764045,0.148912,0.0,0.5,1.0,0.738140,0.333333,0.4,0.009718,0.0,0.285714,0.004501,0.0,0.239484,0.323529,0.057971,0.228074,0.0,0.0,0.004992,0.219512,0.203125,0.000000,0.666667,1.0,0.0,0.75,0.230481,0.594595,0.318182,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.571429,0.414634,0.947368,0.0,0.0,0.293987,0.062500,0.321429,0.184874,0.311688,0.184874,0.311688,0.512605,0.411429,0.098592,0.324786,0.233871,0.266667,0.239669,0.266667,0.256410,0.10,1.00,0.74,0.227679,0.333333,0.288732,0.268908,0.288732,0.268723,1.000000,0.898494,0.000000,0.000000,0.292453,0.450549,0.289720,0.445652,0.618644,0.666667,0.00000,0.000000,0.318966,0.500000,0.324561,0.500000,0.326531,0.095238,1.00,0.74,0.000000,0.000000,0.378378,0.439716,0.378378,0.436620,1.000000,0.718750,0.323529,0.352941,0.003333,0.00346,0.221311,0.245254,0.500000,0.395833,0.201486,0.200251,0.176580,0.298039,0.250000,0.25000,0.201923,0.315789,0.225080,0.333333,0.000000,0.000000,0.046392,0.040816,0.369176,0.397940,0.285714,0.294118,0.371429,0.472222,0.287433,0.330017,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

### Teste

In [ ]:
categorize(test_df)
normalize(test_df)
remove_nan(test_df)

## Definição dos parâmetros para o modelo

As variáveis **encounter_id** e **hospital_id** não foram consideradas para os dados de entrada.

### Treino

In [ ]:
x_training = training_df.loc[:,'age':'solid_tumor_with_metastasis'].to_numpy()
y_training = training_df['diabetes_mellitus'].to_numpy()

print('Amostras de treino:')
print(f' > x_train: {x_training.shape}')
print(f' > y_train: {y_training.shape}')

Amostras de treino:
 > x_train: (117141, 177)
 > y_train: (117141,)


### Teste

In [ ]:
x_test = test_df.loc[:,'age':'solid_tumor_with_metastasis'].to_numpy()

print('Amostras de teste:')
print(f' > x_test: {x_test.shape}')

Amostras de teste:
 > x_test: (13016, 177)


## Treinamento dos modelos

In [ ]:
def show_metric(model):
  score_key = 'roc_auc'
  cv = KFold(n_splits = 3, 
           shuffle = True, 
           random_state=0)

  n_scores = cross_val_score(model, 
                            x_training, y_training, 
                            scoring = score_key, 
                            cv = cv)
  
  print('Validação Cruzada\n')
  print(f'{score_key}: {n_scores}\n')
  print(f'{score_key} (média): {round(np.mean(n_scores), 4)}')
  print(f'{score_key} (desvio padrão): {round(np.std(n_scores), 4)}')

def generate_probs(model, file_name):
  predict_values = model.predict_proba(x_test)
  predict_values = predict_values[:,1]

  submission_df = test_df.copy()
  submission_df = submission_df[['encounter_id']]
  submission_df['diabetes_mellitus'] = predict_values
  submission_df.head()

 
  submission_df.to_csv(file_name, index=False)

### Regressão Logística

In [ ]:
logistic_model = LogisticRegression(penalty='l2', dual=False,
                                    tol=0.0001,
                                    C=1.0, 
                                    fit_intercept=True, 
                                    intercept_scaling=1,
                                    class_weight=None, 
                                    random_state=0, 
                                    solver='newton-cg', 
                                    max_iter=150, 
                                    multi_class='auto', 
                                    verbose=0, 
                                    warm_start=False,
                                    n_jobs=None,
                                    l1_ratio=None)

In [ ]:
logistic_model.fit(x_training, y_training)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=150,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
show_metric(logistic_model)

Validação Cruzada

roc_auc: [0.81903647 0.82184088 0.82664434 0.81825121 0.82106641]

roc_auc (média): 0.8214
roc_auc (desvio padrão): 0.0029


### Árvore de Decisão

In [ ]:
tree_model = DecisionTreeClassifier(criterion='entropy',
                                    max_depth=7,
                                    min_samples_split=2,
                                    random_state=0)

tree_model.fit(x_training, y_training)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
show_metric(tree_model)

Validação Cruzada

roc_auc: [0.80807492 0.80614696 0.81519358 0.81177172 0.81239992]

roc_auc (média): 0.8107
roc_auc (desvio padrão): 0.0032


### Random Forest

In [ ]:
random_forest_model = RandomForestClassifier(n_estimators=30, 
                                             criterion='entropy',
                                             max_depth=10,
                                             min_samples_split=2,
                                             random_state=0)

random_forest_model.fit(x_training, y_training)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
show_metric(random_forest_model)

Validação Cruzada

roc_auc: [0.8242281  0.82368064 0.83279344 0.82366734 0.82620784]

roc_auc (média): 0.8261
roc_auc (desvio padrão): 0.0035


### Soft Voting

In [ ]:
soft_voting_model = VotingClassifier(estimators=[('logistic_model', logistic_model),
                                                 ('tree_model', tree_model)],
                                     voting='soft')

soft_voting_model.fit(x_training, y_training)

VotingClassifier(estimators=[('logistic_model',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=150,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0,
                                                 solver='newton-cg', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('tree_model',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                   

In [ ]:
show_metric(soft_voting_model)

Validação Cruzada

roc_auc: [0.82727646 0.82830347 0.83542278 0.83043433 0.83220984]

roc_auc (média): 0.8307
roc_auc (desvio padrão): 0.0029


### Bagging

In [ ]:
bagging_model = BaggingClassifier(KNeighborsClassifier(n_neighbors=3),
                                  n_estimators=800,
                                  max_samples=100)

bagging_model.fit(x_training, y_training)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='minkowski',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=3, p=2,
                                                      weights='uniform'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=100, n_estimators=800, n_jobs=None,
                  oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

In [ ]:
show_metric(bagging_model)

### Boosting

**Ada Boost com Regressão Logística**

In [ ]:
ada_model = AdaBoostClassifier(LogisticRegression(),
                               n_estimators=300, 
                               learning_rate=0.9)

ada_model.fit(x_training, y_training)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=LogisticRegression(C=1.0, class_weight=None,
                                                     dual=False,
                                                     fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None,
                                                     max_iter=100,
                                                     multi_class='auto',
                                                     n_jobs=None, penalty='l2',
                                                     random_state=None,
                                                     solver='lbfgs', tol=0.0001,
                                                     verbose=0,
                                                     warm_start=False),
                   learning_rate=0.9, n_estimators=300, random_state=None)

### XGBoost

In [ ]:
xgb_model = xgboost.XGBClassifier(booster='gbtree', 
                                  learning_rate=0.05,
                                  n_estimators=400,
                                  max_depth=9, 
                                  use_label_encoder=False)

xgb_model.fit(x_training, y_training)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, use_label_encoder=False, verbosity=1)

### Ensembles de ensembles

In [ ]:
soft_full_model = VotingClassifier(estimators=[('logistic_model', logistic_model),
                                                 ('tree_model', tree_model),
                                                 ('random_forest_model', random_forest_model),
                                                 ('ada_model', ada_model),
                                                 ('xgb_model', xgb_model)], 
                                     voting='soft')

soft_full_model.fit(x_training, y_training)

VotingClassifier(estimators=[('logistic_model',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=150,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0,
                                                 solver='newton-cg', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('tree_model',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=N...
                                            colsample_bytree=1, gamma=0,
                     

### Rede Neural

**Multilayer Perceptron (Scikit-Learn)**

In [ ]:
mlp_model = MLPClassifier(max_iter=600,
                          random_state=0,
                          verbose=0)

mlp_model.fit(x_training, y_training)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=600,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
show_metric(mlp_model)

Validação Cruzada

roc_auc: [0.78109678 0.78319494 0.78175228]

roc_auc (média): 0.782
roc_auc (desvio padrão): 0.0009


**Multilayer Perceptron (Keras)**

In [ ]:
keras_y_training = keras.utils.to_categorical(y_training, num_classes=2)
keras_y_training

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
mlp_keras_model = Sequential()
mlp_keras_model.add(Dense(200, input_shape=(177,), activation='relu'))
mlp_keras_model.add(Dense(100, activation='relu'))
mlp_keras_model.add(Dense(2, activation='softmax'))

mlp_keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               35600     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 55,902
Trainable params: 55,902
Non-trainable params: 0
_________________________________________________________________


In [ ]:
mlp_keras_model.compile(loss='categorical_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

hist = mlp_keras_model.fit(x_training, keras_y_training, epochs=10, verbose=1, callbacks=[callback])

Epoch 1/10
3661/3661 [==============================] - 19s 2ms/step - loss: 0.4304 - accuracy: 0.8083
Epoch 2/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3936 - accuracy: 0.8227
Epoch 3/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3933 - accuracy: 0.8224
Epoch 4/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3911 - accuracy: 0.8237
Epoch 5/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3844 - accuracy: 0.8253
Epoch 6/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3838 - accuracy: 0.8262
Epoch 7/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3833 - accuracy: 0.8270
Epoch 8/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3792 - accuracy: 0.8272
Epoch 9/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.3753 - accuracy: 0.8282
Epoch 10/10
3661/3661 [==============================] - 6s 2ms/step - loss: 0.37

## Comparação entre os modelos

Modelo  | Area Under ROC Curve (média) 
-------------------|------------------
Regressão Logística | 0.8214 
Árvore de Decisão | 0.8107
Random Forest | 0.8261
Soft Voting | 0.8307
Bagging |
Boosting |
XGBoost |
Ensembles de ensembles | 0.84431
Multilayer Perceptron (Scikit-Learn) | 0.81139
Multilayer Perceptron (Keras) | 0.84906